In [52]:
import time
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from math import *
from sklearn import metrics
#
pd.set_option('display.max_columns', 500)  # force pandas to display all columns for better visual inspection
# plot plots inline
%matplotlib inline 

In [ ]:
trn_all = pd.read_csv('../data/train.csv')  # column #0 in our file is index
# trn_1 = pd.read_csv('../data/train_1.csv', index_col=0)

In [ ]:
#selected features and Cut off outliers of Expected >= 69
index=list(trn_all)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18, 23]
trn_new = trn_all[[index[i] for i in my_indices]]

trn_new = trn_new[trn_new['Expected']<69]

In [ ]:
#trn_new

In [ ]:
#combine observations with same ID by using mean
#replace Nan by overall mean
trn_mean = trn_new.groupby(trn_new.Id).agg(['mean', 'median', 'std', 'count'])
trn_mean.columns = ['_'.join(col).strip() for col in trn_mean.columns.values]
trn_mean = trn_mean.drop(['Expected_count', 'Expected_median', 'Expected_std'], axis =1)

In [ ]:
# ignore id's where all Ref vales are NaN
#trn_mean = trn_mean[pd.notnull(trn_mean.Ref_mean)]

In [ ]:
#trn_mean

In [ ]:
index2 = list(trn_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

trn_mean= pd.DataFrame(imp.fit_transform(trn_mean),index = trn_mean.index, columns=index2)

In [ ]:
# reading the MP values for training data
MP_train = pd.read_csv('../data/MP_train.csv')
MP_09 = MP_train.loc[MP_train['Id'].isin(trn_mean.index)]
MP_09.columns = ['Id', 'MP_09']
del MP_09['Id']
MP_09 = MP_09.reset_index()

In [ ]:
#MP_09

In [ ]:
#trn_mean

In [ ]:
# Add MP as a column to trn_mean
trn_mean = trn_mean.reset_index()
trn_mean = pd.concat([trn_mean, MP_09['MP_09']], axis=1)
trn_mean = trn_mean.set_index('Id')

In [ ]:
#trn_mean

In [ ]:
#train and test data preparation
y_trn = np.log1p(trn_mean.loc[:,'Expected_mean'].values)
trn_mean = trn_mean.drop(['Expected_mean'], axis =1)

In [ ]:
trn_mean

In [ ]:
X_trn = trn_mean.loc[:,'minutes_past_mean':'MP_09'].values


# Split data as training and validation set
#[X_trn, X_test, y_trn, y_test] = train_test_split(X, Y, test_size = 0.3)


In [ ]:
X_trn

In [ ]:
clf = RandomForestRegressor(n_estimators=100,n_jobs=4)  # NOTE: n_jobs=-1 will use all of your cores, set to a prefered number

In [ ]:
#train model
t = time.time()
clf.fit(X_trn, y_trn)
print time.time()-t

In [ ]:
t

In [ ]:
#test performance
#y_predict = clf.predict(X_test)
#score = metrics.mean_absolute_error(y_test, np.exp(y_predict)-1) #MAE

In [ ]:
#print score

In [ ]:
#generate test result
test = pd.read_csv('../data/test.csv')

#selected features and Cut off outliers of Expected >= 69
index=list(test)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18]
test_new = test[[index[i] for i in my_indices]]


#combine observations with same ID by using mean
#replace Nan by overall mean
test_mean = test_new.groupby(test_new.Id).agg(['mean', 'median', 'std', 'count'])
test_mean.columns = ['_'.join(col).strip() for col in test_mean.columns.values]

# reading the MP values for testing data
MP_test = pd.read_csv('../data/MP_r_09.csv')
MP_te_09 = MP_test.loc[MP_test['Id'].isin(test_mean.index)]
MP_te_09.columns = ['Id', 'MP_09']
del MP_te_09['Id']
MP_te_09 = MP_te_09.reset_index()

In [ ]:
# Add MP as a column to test_mean
test_mean = test_mean.reset_index()
test_mean = pd.concat([test_mean, MP_te_09['MP_09']], axis=1)
test_mean = test_mean.set_index('Id')

# Imputing with mean values
index2 = list(test_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
test_mean= pd.DataFrame(imp.fit_transform(test_mean),index=test_mean.index,columns=index2)

test_X =test_mean.loc[:,'minutes_past_mean':'MP_09'].values

test_y_predict = np.exp(clf.predict(test_X))-1

In [ ]:
#generate output file
#0.75 from prediction and 0.25 from marshall palmer
marshall = pd.read_csv('../data/MP_r_09.csv')

test_result_exist = pd.DataFrame()
test_result_exist['Id'] = test_mean.index
test_result_exist['Expected'] = test_y_predict

test_result = pd.DataFrame()
test_result['Id'] = test['Id'].unique()
test_result = pd.merge(test_result, test_result_exist, how='left', on=['Id'], sort=True)
test_result.loc[test_result['Expected'].isnull(), 'Expected'] = marshall.loc[test_result['Expected'].isnull(), 'Expected']
test_result.loc[test_result['Expected'].notnull(), 'Expected'] = 0.75*test_result.loc[test_result['Expected'].notnull(), 'Expected']+0.25*marshall.loc[test_result['Expected'].notnull(), 'Expected']

test_result.to_csv('../data/pks_result_v5.csv', index=False)

In [ ]:
test_result